<a href="https://colab.research.google.com/github/DriftKing-byte/QA-Chatbot/blob/main/NLP_Transformers_Clinical_Note_Summarizer_Via_Gen_AI_(tiny_llama).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In NLP latest architecture is Transformer based architectures There's three types. Encoder-only models like BERT, decoder-only models like GPT and llama. There's encoder-decoder architectures as well: Encoder-decoder models can be implemented using various neural network architectures, and their names often reflect the type of network used or the specific application. Here are some key examples:
1. Transformer-based Encoder-Decoder Models:
These models are based on the Transformer architecture, which has significantly advanced the field of natural language processing.
T5 (Text-to-Text Transfer Transformer): T5 is a Transformer-based encoder-decoder model that treats all NLP tasks as text-to-text problems. It's widely used for tasks like machine translation, summarization, and question answering.
BART (Bidirectional and Auto-Regressive Transformer): BART is another Transformer-based encoder-decoder model that excels at denoising sequence-to-sequence tasks. It's particularly effective for text generation and summarization.
Pegasus (Pre-training with Extracted Gap-sentences for Abstractive Summarization): Pegasus is a Transformer-based encoder-decoder model specifically designed for abstractive summarization, where it generates summaries by focusing on key sentences extracted from the input text.
MT5 (Massively Multilingual Text-to-Text Transformer): MT5 is a multilingual variant of T5, trained on a large corpus of text in various languages.
FLAN-T5 (Scaling Instruction-Finetuned Language Models): FLAN-T5 is an extension of T5 that has been finetuned on a wide range of tasks and instructions to improve its generalization capabilities.
Code-T5: This is a variant of T5 designed specifically for code understanding and generation.
UL2 (Unifying Language Learning Paradigms): UL2 is another Transformer-based encoder-decoder model with a unified approach to language learning.
FLAN-UL2: This is a finetuned version of UL2 with improved performance on various tasks.
EdgeFormer: A Transformer-based encoder-decoder model designed for efficient seq2seq generation on devices with limited resources.
2. Models Utilizing Encoder-Decoder Architecture for Specific Tasks:
Encoder-decoder architectures can also be used as components within larger models designed for specific tasks:
VisionEncoderDecoderModel: This model initializes an image-to-text model with a pretrained vision model (like ViT) as the encoder and a pretrained language model (like BERT or GPT2) as the decoder. This allows it to perform tasks like image captioning and optical character recognition (OCR).
TrOCR (Transformer-based Optical Character Recognition): TrOCR is a specific instance of the VisionEncoderDecoderModel architecture, fine-tuned for OCR.
Note: Encoder-decoder architecture is a framework, and specific implementations can vary in their internal network structure (RNN, CNN, Transformer) and pre-training objectives.

LangChain can be used for app development. Web-side of app development is still done with FastAPI. If you want to include agents then Langgraph is used. Fine-tuning of the model for best hyperparameters can be done using LoRA or QLoRA. LoRA is where memory is a constraint but want to maintain high precision. QLoRA (quantized lora) is used to optimize memory efficiency in comprise for a minimal loss in performance.

Llama 2 and 3 and mistral ai were attempted but did not work with system ram and gpu usage constraints. Tiny Llama was finally leveraged on a dataset that contained two columns: one with a full clinical note and the other with the summary written by a healthcare provider. The llm was given the task to summarize a given synthetic clinical note by chatgpt4. The result was an accurate response with extra unnecessary details

In [ ]:
!pip install triton==2.1.0 bitsandbytes==0.41.0 peft==0.7.0 transformers==4.38.2 accelerate==0.30.0 trl==0.4.7

In [ ]:
!pip install huggingface_hub
!pip install numpy
import numpy as np

In [ ]:
#!pip install triton
import torch
from transformers import (AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments, pipeline)
from transformers.generation import LogitsProcessorList, TopKLogitsWarper, TopPLogitsWarper
from trl import SFTTrainer
from peft import LoraConfig
from datasets import load_dataset


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py", line 37, in <module>
    ColabKernelApp.launch_instance()
  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelapp.py", line 712, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.11/dist-package

Llama model requires too much system ram and is crashing which is the reason for using tiny llama.

In [ ]:
llama_model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    # Remove quantization_config
).to('cpu') # Explicitly move the model to CPU
llama_model.config.use_cache = False
llama_model.config.pretraining_tp = 1

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


The below code is specifically for GPU. Bitsandbytes config with quantization is for gpu. The above code is for CPU.

In [ ]:
###llama_model = AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path= "aboonaji/llama2finetune-v3",
                                                  # quantization_config= BitsAndBytesConfig(load_in_4bit = True,
                                                   # bnb_4bit_compute_dtype = getattr(torch, "float16"), bnb_4bit_quant_type = "nf4"))
##llama_model.config.use_cache = False
##llama_model.config.pretraining_tp = 1

In [ ]:
llama_tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path= "TinyLlama/TinyLlama-1.1B-Chat-v1.0", trust_remote_code = True)
llama_tokenizer.pad_token = llama_tokenizer.eos_token
llama_tokenizer.padding_side = "right"

In [ ]:
training_arguments = TrainingArguments(output_dir= "./results", per_device_train_batch_size = 1, max_steps = 4)

In [ ]:
!pip install -U datasets huggingface_hub fsspec


load_dataset("geekdom/clinical_data")
print(load_dataset)

  Using cached fsspec-2025.5.1-py3-none-any.whl.metadata (11 kB)


refined_clinical_data.jsonl:   0%|          | 0.00/53.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/19756 [00:00<?, ? examples/s]

<function load_dataset at 0x7a3e00615da0>


In [ ]:
dataset = load_dataset("geekdom/clinical_data", split="train")

def format_for_sft(example):
    return {
        "text": f"Summarize:\n{example['prompt']}\n\nSummary:\n{example['response']}"
    }

# Apply formatting
dataset = dataset.map(format_for_sft)

Map:   0%|          | 0/19756 [00:00<?, ? examples/s]

In [ ]:

llama_sft_trainer = SFTTrainer(model = llama_model,
                               args = training_arguments,
                               train_dataset = dataset,
                               tokenizer = llama_tokenizer,
                               peft_config = LoraConfig(task_type = "CAUSAL_LM", r = 64, lora_alpha = 16, lora_dropout = 0.1),
                               dataset_text_field = "text")

/usr/local/lib/python3.11/dist-packages/trl/trainer/sft_trainer.py:159: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/19756 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/accelerate/accelerator.py:446: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [ ]:
llama_sft_trainer.train()

<IPython.core.display.Javascript object>

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: saiashutosh-c (saiashutosh-c-gmu) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss


TrainOutput(global_step=4, training_loss=1.6238625049591064, metrics={'train_runtime': 11.3568, 'train_samples_per_second': 0.352, 'train_steps_per_second': 0.352, 'total_flos': 16197573070848.0, 'train_loss': 1.6238625049591064, 'epoch': 0.0})

In [ ]:
user_prompt = (
    """Below is an instruction that describes a task. Write a response that appropriately completes the request.\n
    Make sure to use line breaks when appropriate.\n
    ### Instruction:\n
    Below is a clinical note.\n
    Your task is to summarize the clinical note in that you describe the patient's course of progression.\n
    ### Input:\n
    Hospital Course: The patient was admitted to the ICU six days after testing positive for COVID-19 due to worsening respiratory distress.\n
    Early physical therapy and prone positioning led to improved oxygen saturation from 85% to 94%. After five days of supportive care and rehabilitation,\n
    the patient was transferred to the general ward and continued progressing with assisted ambulation and breathing exercises.\n
    Discharge Condition: At discharge, the patient was stable, breathing comfortably on room air, and able to walk short distances with minimal assistance.\n
    Oxygen saturation and respiratory rate were within normal limits.\n
    ### Response:"""
)
#text_generation_pipeline = pipeline(task = "text-generation", model = llama_model, tokenizer = llama_tokenizer, max_length = 200)
#model_answer =  text_generation_pipeline(user_prompt)
#print(model_answer[0]['generated_text'])
inputs = llama_tokenizer(user_prompt, return_tensors="pt", padding = True, truncation = True)

# Ensure tensors are on CPU if you're CPU-only
inputs = {key: val.to("cpu") for key, val in inputs.items()}

with torch.no_grad():
    outputs = llama_model.generate(
        **inputs,
        max_new_tokens=200,
        do_sample=True,
        temperature=0.7,
        top_p=0.95
    )

response = llama_tokenizer.decode(outputs[0], skip_special_tokens=True)
print(response)

Below is an instruction that describes a task. Write a response that appropriately completes the request.

    Make sure to use line breaks when appropriate.

    Start with the response without any extra detail before.

    ### Instruction:

    Below is a clinical note.
 
    Your task is to summarize the clinical note in that you describe the patient's course of progression.

    ### Input:

    Hospital Course: The patient was admitted to the ICU six days after testing positive for COVID-19 due to worsening respiratory distress.
 
    Early physical therapy and prone positioning led to improved oxygen saturation from 85% to 94%. After five days of supportive care and rehabilitation,
 
    the patient was transferred to the general ward and continued progressing with assisted ambulation and breathing exercises.
 
    Discharge Condition: At discharge, the patient was stable, breathing comfortably on room air, and able to walk short distances with minimal assistance.
 
    Oxygen sat

# Result:

Below is the input given to the model. The model produces a decent answer but the answer depends exremely on the instructions given. The answer is accurate but lacks precision because extra details before or after the response are sometimes given. The response is unique each generation of response.

# Given Input

Hospital Course:

The patient was admitted to the ICU six days after testing positive for COVID-19 due to worsening respiratory distress. Early physical therapy and prone positioning led to improved oxygen saturation from 85% to 94%. After five days of supportive care and rehabilitation, the patient was transferred to the general ward and continued progressing with assisted ambulation and breathing exercises.

Discharge Condition:

At discharge, the patient was stable, breathing comfortably on room air, and able to walk short distances with minimal assistance. Oxygen saturation and respiratory rate were within normal limits.